<a href="https://colab.research.google.com/github/modulabs-personalab/sae-experiments/blob/main/experiments/GPT2_Pretrained_dictionary_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers transformer-lens
!pip install transformers==4.31.0 transformer-lens==1.7.0
!pip install git+https://github.com/openai/sparse_autoencoder.git

Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 818.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta

In [2]:
import gc
gc.collect()

124

In [83]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

In [3]:
import torch
import blobfile as bf
import transformer_lens
import sparse_autoencoder
from difflib import SequenceMatcher

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2", center_writing_weights=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [150]:
# Load pretrained SAE

layer_index = 8
location = "resid_post_mlp"
device = next(model.parameters()).device

transformer_lens_loc = {
    "mlp_post_act": f"blocks.{layer_index}.mlp.hook_post",
    "resid_delta_attn": f"blocks.{layer_index}.hook_attn_out",
    "resid_post_attn": f"blocks.{layer_index}.hook_resid_mid",
    "resid_delta_mlp": f"blocks.{layer_index}.hook_mlp_out",
    "resid_post_mlp": f"blocks.{layer_index}.hook_resid_post",
}[location]

sae_param_path = sparse_autoencoder.paths.v5_128k(location, layer_index)
print("Weights url:", sae_param_path)

with bf.BlobFile(sae_param_path, mode="rb") as f:
    state_dict = torch.load(f)
    autoencoder = sparse_autoencoder.Autoencoder.from_state_dict(state_dict)
    autoencoder.to(device)

print(autoencoder)

Weights url: az://openaipublic/sparse-autoencoder/gpt2-small/resid_post_mlp_v5_128k/autoencoders/8.pt
Autoencoder(
  (encoder): Linear(in_features=768, out_features=131072, bias=False)
  (activation): TopK(
    (postact_fn): ReLU()
  )
  (decoder): Linear(in_features=131072, out_features=768, bias=False)
)


In [235]:
prompt = "he is such a "
tokens = model.to_tokens(prompt)  # (1, n_tokens)
print("tokens.shape", tokens.shape)

with torch.no_grad():
    logits, activation_cache = model.run_with_cache(tokens, remove_batch_dim=True)

tokens.shape torch.Size([1, 6])


In [236]:
def get_remarkable_features(prompt):
  tokens = model.to_tokens(prompt)  # (1, n_tokens)
  with torch.no_grad():
    logits, activation_cache = model.run_with_cache(tokens, remove_batch_dim=True)
    input_tensor = activation_cache[transformer_lens_loc]
    latent_activations, _ = autoencoder.encode(input_tensor)
    top_values = []
    top_feature_indicies = []
    values, indicies = torch.topk(latent_activations[-1], 10)
    return indicies.tolist()

print(get_remarkable_features("he is good guy"))
print(get_remarkable_features("he is good guy, i like him"))
print(get_remarkable_features("i like him, he is really good guy"))
print(get_remarkable_features("i am his best friend."))
print(get_remarkable_features("he is"))
print(get_remarkable_features("he is sucks and fucking stupid idiot"))
print(get_remarkable_features("sucks and fucking stupid idiot"))
print(get_remarkable_features("i hate him. he is ugly and stupid"))
print(get_remarkable_features("he is my friend"))
print(get_remarkable_features("he is my enemy"))

[97009, 67809, 4057, 28212, 121329, 127471, 4644, 35056, 16840, 43408]
[49782, 28981, 112479, 14506, 70588, 127471, 64615, 19835, 43408, 4057]
[97009, 28212, 127471, 67809, 4057, 20744, 127761, 115311, 16840, 74171]
[99456, 57176, 7830, 28732, 737, 123834, 19983, 114859, 90753, 122895]
[100638, 53260, 73184, 81637, 130253, 36695, 9736, 73157, 32505, 91367]
[62556, 79394, 4057, 78339, 127471, 4644, 16840, 17873, 109505, 86309]
[62556, 79394, 4057, 78339, 4644, 31786, 69689, 24128, 35401, 11982]
[40814, 11982, 59378, 12947, 67518, 127471, 113485, 127761, 43408, 109438]
[29316, 107790, 4057, 64692, 127471, 12066, 54902, 100087, 118706, 85974]
[121621, 64692, 4057, 107790, 36695, 9736, 127471, 13059, 85974, 111159]


In [190]:
# 텍스트 생성
max_tokens = 50
temperature = 0.0
seed = 123
torch.manual_seed(seed)

prompt = "he is such a"

# Generate text without modifying activations
generated_text_without_hook = model.generate(prompt,
                                             max_new_tokens=max_tokens,
                                             temperature=temperature,
                                             top_k=None,  # top_k를 None으로 설정하여 temperature만 사용
                                             top_p=None)  # top_p를 None으로 설정하여 temperature만 사용

print("prompt:", prompt)
print("output:", generated_text_without_hook)

  0%|          | 0/50 [00:00<?, ?it/s]

prompt: he is such a
output: he is such a good person, he is such a good person, he is such a good person, he is such a good person, he is such a good person, he is such a good person, he is such a good person, he is such a good


In [191]:
def get_feature(indicies):
  vector = np.zeros(131072)
  vector[indicies] = 1
  input_tensor = torch.tensor(vector, dtype=torch.float32)
  with torch.no_grad():
    feat = autoencoder.decoder(input=input_tensor)
    return feat

get_feature([62663, 24881, 47801]).shape

torch.Size([768])

In [234]:
target_layer = model.blocks[layer_index]

# negative_feature = get_feature([97009, 28212, 67809, 29316, 12066, 54902, 100087, 118706])
# positive_feature = get_feature([4644, 62556, 79394, 69689, 86309]) # I'm not sure if he's a good guy or not.
# positive_feature = get_feature([62556, 79394, 69689, 86309]) # he is such a stupid, stupid
# positive_feature = get_feature([62556, 79394, 86309]) # he is such a fool. I am a fool.I am a fool.
# positive_feature = get_feature([62556, 86309]) # he is such a good person. I am a little bit of a jerk.
positive_feature = get_feature([62556, 79394, 86309, 69689])
# positive_feature = get_feature([62556, 79394]) # I'm not sure if he's a good guy, but he's a good guy.
# positive_feature = get_feature([62556, 79394, 86309]) # I'm not sure if he's a good guy or not.
# positive_feature = get_feature([79394, 69689, 86309]) # he is such a good person,

# +62556 : coward -> fool
# +86309 : not sure -> sure
# +79394 : target
# +69689 : focus target

def activation_patching(layer, input, output):
    if layer == target_layer: # 해당 레이어인지 확인
        return output + (positive_feature*20) # - (negative_feature * 5)
    return output

hook_handle = target_layer.register_forward_hook(activation_patching)

# Generate text without modifying activations
max_tokens = 50
temperature = 0.7
seed = 123
torch.manual_seed(seed)
generated_text_without_hook = model.generate(prompt,
                                             max_new_tokens=max_tokens,
                                             temperature=temperature,
                                             top_k=None,  # top_k를 None으로 설정하여 temperature만 사용
                                             top_p=None)  # top_p를 None으로 설정하여 temperature만 사용

print("prompt:", prompt)
print("output:", generated_text_without_hook)
hook_handle.remove()

  0%|          | 0/50 [00:00<?, ?it/s]

prompt: he is such a
output: he is such a shit I will never be able to do it again


did i not say i don't want to do it? i just said i don't want to do it


it sucks to smile when so many people are just trying to think about
